In [1]:
import pandas as pd
import time
from os.path import join
import json 
import numpy as np 
from tqdm import tqdm
import seaborn as sns
from os import listdir, path
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import missingno as msno
from matplotlib.colors import ListedColormap
import plotly.graph_objs as go
# make a train test split 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_parquet('../data/preprocessed_data/training_data/df_failures_telemetry.parquet')

In [4]:
df.value_counts('failure')

failure
0    654052
2       187
1       153
4       138
3       103
dtype: int64

In [5]:
df.query('failure != 0')#.value_counts('model')

,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary
96,2015-01-05 06:00:00,1,179.303153,499.777962,111.833028,52.383097,4,1
1536,2015-03-06 06:00:00,1,198.257975,456.862342,89.333995,38.671900,1,1
2616,2015-04-20 06:00:00,1,180.050801,346.362480,105.661164,39.218055,2,1
4056,2015-06-19 06:00:00,1,187.673963,493.005160,105.334392,53.963961,4,1
5856,2015-09-02 06:00:00,1,144.094532,409.380150,106.720871,57.454990,4,1
...,...,...,...,...,...,...,...,...
646269,2015-07-17 06:00:00,99,173.273231,490.753609,135.766664,37.678738,3,1
646629,2015-08-01 06:00:00,99,147.542660,471.754736,104.324517,42.983190,2,1
647349,2015-08-31 06:00:00,99,189.337240,450.150198,103.996522,40.219528,3,1
649095,2015-02-12 06:00:00,100,226.666914,484.516473,97.521897,53.129437,1,1


In [6]:
df.iloc[1500:1540]

,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary
1500,2015-03-04 18:00:00,1,186.260099,493.983905,104.296202,44.468203,0,0
1501,2015-03-04 19:00:00,1,171.671520,360.736832,101.886029,39.946633,0,0
1502,2015-03-04 20:00:00,1,188.911905,383.292730,102.418570,37.633302,0,0
1503,2015-03-04 21:00:00,1,210.473519,472.531431,112.533270,40.620846,0,0
1504,2015-03-04 22:00:00,1,183.137035,441.870156,92.064710,34.676932,0,0
1505,2015-03-04 23:00:00,1,190.978003,488.490140,114.346162,47.313941,0,0
1506,2015-03-05 00:00:00,1,201.759705,403.020399,92.791004,32.827544,0,0
1507,2015-03-05 01:00:00,1,166.184379,398.262189,96.867771,32.686649,0,0
1508,2015-03-05 02:00:00,1,169.459535,468.067109,110.281958,45.230072,0,0
1509,2015-03-05 03:00:00,1,176.087203,506.733460,93.488252,37.642103,0,0


In [7]:
ventana_features = 40
ventana_falla = 10

def seleccionar_ventanas(ventana_features,ventana_falla):

    df_ventana = pd.DataFrame( )
    for id_falla,i_falla in enumerate(df.query('failure != 0').index):
        shift = np.random.randint(1,ventana_falla+1)
        df_temp = df.iloc[i_falla -ventana_features + shift: i_falla + shift].copy()
        df_temp = df_temp.iloc[0:ventana_features-ventana_falla]
        df_temp['id_falla'] = id_falla
        df_temp.iloc[ventana_features-ventana_falla-1,6] = df.iloc[i_falla,6]
        df_temp.iloc[ventana_features-ventana_falla-1,7] = df.iloc[i_falla,7]
        df_ventana = pd.concat([df_ventana,df_temp]) 
    return df_ventana


df_ventana = seleccionar_ventanas(ventana_features,ventana_falla)


In [8]:
df_ventana.value_counts('failure')

failure
0    16848
2      188
1      153
4      138
3      103
dtype: int64

,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary,id_falla
62,2015-01-03 20:00:00,1,163.783704,520.582694,97.160984,56.442488,0,0,0
63,2015-01-03 21:00:00,1,167.254238,382.541315,104.485227,39.273567,0,0,0
64,2015-01-03 22:00:00,1,176.105841,428.325822,107.266872,52.364675,0,0,0
65,2015-01-03 23:00:00,1,179.369497,374.127148,87.737435,49.153972,0,0,0
66,2015-01-04 00:00:00,1,195.173086,436.173620,103.833238,49.947102,0,0,0
...,...,...,...,...,...,...,...,...,...
654130,2015-09-10 01:00:00,100,214.012906,455.727429,100.013031,40.902838,0,0,580
654131,2015-09-10 02:00:00,100,197.067317,513.200942,95.422995,40.437980,0,0,580
654132,2015-09-10 03:00:00,100,185.310093,462.133184,105.136787,35.817124,0,0,580
654133,2015-09-10 04:00:00,100,186.687188,443.453437,92.263949,45.004588,0,0,580


In [41]:

def seleccionar_ventanas_sin_fallas(ventana_features,ventana_falla,n):

    df_ventana = pd.DataFrame( )
    for ix,id  in enumerate( range(1000,len(df)-1000,800)):
        df_temp = df.iloc[ix:ix+ventana_features].copy()
        if df_temp['failure'].max()!=0:
            continue
        df_temp['id_falla'] = ix
        df_ventana = pd.concat([df_ventana,df_temp])


    if len(df_ventana['id_falla'].unique()) < n:
        return df_ventana 
    else:
        # make a random sample of n ids  
        ids = np.random.choice(df_ventana['id_falla'].unique(),n,replace=False)

        df_ventana = df_ventana.query('id_falla in @ids')
        
        return df_ventana
    

    

In [47]:
df_ventana  = seleccionar_ventanas_sin_fallas(ventana_features,ventana_falla,700)
df_ventana


,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary,id_falla
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0,0,0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0,0,0
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,0,0,0
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,0,0,0
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,0,0,0
...,...,...,...,...,...,...,...,...,...
850,2015-02-05 16:00:00,1,158.799503,441.730202,96.632135,35.767776,0,0,815
851,2015-02-05 17:00:00,1,173.161721,401.314578,102.943588,46.266280,0,0,815
852,2015-02-05 18:00:00,1,157.272353,463.671477,96.702439,43.055695,0,0,815
853,2015-02-05 19:00:00,1,181.883971,496.708290,91.632890,33.542046,0,0,815


(700,)

,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary,id_falla
5816,2015-08-31 14:00:00,1.0,154.549723,432.808321,98.728875,55.246831,0.0,0.0,4.0
5817,2015-08-31 15:00:00,1.0,143.911321,482.729506,111.432430,43.475987,0.0,0.0,4.0
5818,2015-08-31 16:00:00,1.0,171.687714,394.202261,90.972853,42.354253,1.0,0.0,4.0
5819,2015-08-31 17:00:00,1.0,164.648960,448.719026,97.926104,53.446291,0.0,0.0,4.0
5820,2015-08-31 18:00:00,1.0,172.996607,421.424571,79.267492,40.510669,0.0,0.0,4.0
5821,2015-08-31 19:00:00,1.0,150.968899,484.243200,110.642836,58.889644,0.0,0.0,4.0
5822,2015-08-31 20:00:00,1.0,180.248624,406.823174,96.406769,56.275566,0.0,0.0,4.0
5823,2015-08-31 21:00:00,1.0,174.102013,481.779100,101.953891,50.037437,0.0,0.0,4.0
5824,2015-08-31 22:00:00,1.0,162.250917,549.741973,118.961608,43.489968,0.0,0.0,4.0
5825,2015-08-31 23:00:00,1.0,168.963619,454.954788,95.816718,57.328544,0.0,0.0,4.0
